In [ ]:
# I have not performed any data preprocessing in this example . 
#Idea is to show the difference in working of both tunning methods 

In [27]:
# importing necessary libraries
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

In [28]:
import warnings
warnings.filterwarnings("ignore")


In [29]:
df=pd.read_csv('train.csv')

In [30]:

# Splitting the data into features (X) and target (y)
X = df[['trip_duration', 'distance_traveled', 'num_of_passengers', 'tip', 'miscellaneous_fees', 'surge_applied']]
y = df['total_fare']

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
# Defining the parameter grid
param_grid = {
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 3, 5, 7],
}

# Performing GridSearchCV
grid_search = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Getting the best model from GridSearchCV
best_model_grid = grid_search.best_estimator_
depth_grid = best_model_grid.get_depth()
params_grid = best_model_grid.get_params()

# Predicting on train and test sets using the best model from GridSearchCV
y_train_pred_grid = best_model_grid.predict(X_train)
y_test_pred_grid = best_model_grid.predict(X_test)

# Calculating MAE and RMSE for train and test sets using the best model from GridSearchCV
train_mae_grid = mean_absolute_error(y_train, y_train_pred_grid)
test_mae_grid = mean_absolute_error(y_test, y_test_pred_grid)
train_rmse_grid = mean_squared_error(y_train, y_train_pred_grid, squared=False)
test_rmse_grid = mean_squared_error(y_test, y_test_pred_grid, squared=False)

# Creating a DataFrame to store the results
results = pd.DataFrame({'Method': ['Decision Tree (GridSearchCV)'],
                        'Train MAE': [train_mae_grid],
                        'Test MAE': [test_mae_grid],
                        'Train RMSE': [train_rmse_grid],
                        'Test RMSE': [test_rmse_grid],
                        'Depth': [depth_grid],
                        'Fitted Parameters': [params_grid]})


In [32]:
# Performing RandomizedSearchCV
randomized_search = RandomizedSearchCV(DecisionTreeRegressor(random_state=42), param_distributions=param_grid, n_iter=10, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)
randomized_search.fit(X_train, y_train)

# Getting the best model from RandomizedSearchCV
best_model_random = randomized_search.best_estimator_
depth_random = best_model_random.get_depth()
params_random = best_model_random.get_params()

# Predicting on train and test sets using the best model from RandomizedSearchCV
y_train_pred_random = best_model_random.predict(X_train)
y_test_pred_random = best_model_random.predict(X_test)

# Calculating MAE and RMSE for train and test sets using the best model from RandomizedSearchCV
train_mae_random = mean_absolute_error(y_train, y_train_pred_random)
test_mae_random = mean_absolute_error(y_test, y_test_pred_random)
train_rmse_random = mean_squared_error(y_train, y_train_pred_random, squared=False)
test_rmse_random = mean_squared_error(y_test, y_test_pred_random, squared=False)

# Adding the results of RandomizedSearchCV to the DataFrame
results = results.append({'Method': 'Decision Tree (RandomizedSearchCV)',
                          'Train MAE': train_mae_random,
                          'Test MAE': test_mae_random,
                          'Train RMSE': train_rmse_random,
                          'Test RMSE': test_rmse_random,
                          'Depth': depth_random,
                          'Fitted Parameters': params_random}, ignore_index=True)


In [33]:
results

,Method,Train MAE,Test MAE,Train RMSE,Test RMSE,Depth,Fitted Parameters
0,Decision Tree (GridSearchCV),4.303690,7.184586,29.938416,48.445506,20,"{'ccp_alpha': 0.0, 'criterion': 'squared_error..."
1,Decision Tree (RandomizedSearchCV),4.749561,7.038871,31.903498,44.320716,20,"{'ccp_alpha': 0.0, 'criterion': 'squared_error..."
